In [1]:
import findspark
import re
findspark.init('/home/cse587/spark-2.4.0-bin-hadoop2.7')
import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
import pandas as pd
from nltk.corpus import stopwords
import nltk 
from pyspark.ml.feature import RegexTokenizer, StopWordsRemover, CountVectorizer
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoder,StringIndexer, VectorAssembler
import numpy as np
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.classification import SVMWithSGD, SVMModel
from pyspark.mllib.linalg import Vector as MllibVector, Vectors as MLLibVectors

In [2]:
sc = pyspark.SparkContext()

In [3]:
spark = SparkSession \
    .builder \
    .appName("PA3") \
    .config("spark.some.config.option","some-value") \
    .getOrCreate()

In [4]:
#read in data
df = pd.read_csv("train.csv")
df = spark.createDataFrame(df)

In [5]:
df =df.withColumn('plot',lower(regexp_replace(df['plot'],"[^a-zA-Z ]","")))

In [6]:
regexToken = RegexTokenizer(inputCol='plot',outputCol='tokens', pattern=" ")

In [7]:
df = regexToken.transform(df)

In [8]:
stop = stopwords.words('english')

In [9]:
stopWordsRemover = StopWordsRemover(inputCol = 'tokens',outputCol = 'cleaned').setStopWords(stop)

In [10]:
df = stopWordsRemover.transform(df)

In [11]:
counts = CountVectorizer(inputCol= 'cleaned',outputCol= 'features',minDF=10)

In [12]:
model = counts.fit(df)

In [13]:
df = model.transform(df)

In [14]:
 df =df.drop('plot','tokens')

In [15]:
labels = df.select('genre')

In [16]:
labels= labels.rdd.flatMap(lambda x:x)

In [17]:
feat = df.select('features')
feat = feat.rdd.flatMap(lambda x:x)

In [18]:
feats = feat.collect()

In [19]:
df.printSchema()

root
 |-- movie_id: long (nullable = true)
 |-- movie_name: string (nullable = true)
 |-- genre: string (nullable = true)
 |-- cleaned: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- features: vector (nullable = true)



In [20]:
movie_ids = df.select('movie_id')
movie_ids = movie_ids.rdd
movie_ids = movie_ids.flatMap(lambda x:x)
movie_ids = movie_ids.collect()


In [21]:
labels = df.select('genre')


In [25]:
def checkDrama(x):
    if 'Drama' in x:
        return 1
    else:
        return 0

checkDrama1 = udf(checkDrama)
df = df.withColumn("Drama",checkDrama1('genre'))

In [27]:
def checkComedy(x):
    if 'Comedy' in x:
        return 1
    else:
        return 0

checkComedy1 = udf(checkComedy)
df = df.withColumn("Comedy",checkComedy1('genre'))

In [32]:
def checkRomanceFilm(x):
    if 'Romance Film' in x:
        return 1
    else:
        return 0

checkRomanceFilm1 = udf(checkRomanceFilm)
df = df.withColumn("Romance Film",checkRomanceFilm1('genre'))

In [34]:
def checkThriller(x):
    if 'Thriller' in x:
        return 1
    else:
        return 0

checkThriller1 = udf(checkThriller)
df = df.withColumn("Thriller",checkThriller1('genre'))

In [35]:
def checkAction(x):
    if 'Action' in x:
        return 1
    else:
        return 0

checkAction1 = udf(checkAction)
df = df.withColumn("Action",checkAction1('genre'))

In [36]:
def checkWorldCinema(x):
    if 'World cinema' in x:
        return 1
    else:
        return 0

checkWorldCinema1 = udf(checkWorldCinema)
df = df.withColumn("World Cinema",checkWorldCinema1('genre'))

In [37]:
def checkCrimeFiction(x):
    if 'Crime Fiction' in x:
        return 1
    else:
        return 0

checkCrimeFiction1 = udf(checkCrimeFiction)
df = df.withColumn("Crime Fiction",checkCrimeFiction1('genre'))

In [38]:
def checkHorror(x):
    if 'Horror' in x:
        return 1
    else:
        return 0

checkHorror1 = udf(checkHorror)
df = df.withColumn("Horror",checkHorror1('genre'))

In [39]:
def checkBlackWhite(x):
    if 'Black-and-white' in x:
        return 1
    else:
        return 0

checkBlackWhite1 = udf(checkBlackWhite)
df = df.withColumn("Black-and-White",checkBlackWhite1('genre'))

In [40]:
def checkIndie(x):
    if 'Indie' in x:
        return 1
    else:
        return 0

checkIndie1 = udf(checkIndie)
df = df.withColumn("Indie",checkIndie1('genre'))

In [42]:
def checkActionAdventure(x):
    if 'Action/Adventure' in x:
        return 1
    else:
        return 0

checkActionAdventure1 = udf(checkActionAdventure)
df = df.withColumn("Action/Adventure",checkActionAdventure1('genre'))

In [43]:
def checkAdventure(x):
    if 'Adventure' in x:
        return 1
    else:
        return 0

checkAdventure1 = udf(checkAdventure)
df = df.withColumn("Adventure",checkAdventure1('genre'))

In [44]:
def checkFamilyFilm(x):
    if 'Family Film' in x:
        return 1
    else:
        return 0

checkFamilyFilm1 = udf(checkFamilyFilm)
df = df.withColumn("Family Film",checkFamilyFilm1('genre'))

In [46]:
def checkShortFilm(x):
    if 'Short Film' in x:
        return 1
    else:
        return 0

checkShortFilm1 = udf(checkShortFilm)
df = df.withColumn("Short Film",checkShortFilm1('genre'))

In [47]:
def checkRomanticDrama(x):
    if 'Romantic drama' in x:
        return 1
    else:
        return 0

checkRomanticDrama1 = udf(checkRomanticDrama)
df = df.withColumn("Romantic drama",checkRomanticDrama1('genre'))

In [49]:
def checkAnimation(x):
    if 'Animation' in x:
        return 1
    else:
        return 0

checkAnimation1 = udf(checkAnimation)
df = df.withColumn("Animation",checkAnimation1('genre'))

In [50]:
def checkMusical(x):
    if 'Musical' in x:
        return 1
    else:
        return 0

checkMusical1 = udf(checkMusical)
df = df.withColumn("Musical",checkMusical1('genre'))

In [51]:
def checkScienceFiction(x):
    if 'Science Fiction' in x:
        return 1
    else:
        return 0

checkScienceFiction1 = udf(checkScienceFiction)
df = df.withColumn("Science Fiction",checkScienceFiction1('genre'))

In [52]:
def checkMystery(x):
    if 'Mystery' in x:
        return 1
    else:
        return 0

checkMystery1 = udf(checkMystery)
df = df.withColumn("Mystery",checkMystery1('genre'))

In [53]:
def checkRomanticComedy(x):
    if 'Romantic comedy' in x:
        return 1
    else:
        return 0

checkRomanticComedy1 = udf(checkRomanticComedy)
df = df.withColumn("Romantic comedy",checkRomanticComedy1('genre'))